# Mphasis HyperGraf Movie Reviews Sentiment Analyzer

Mphasis HyperGraf Movie Reviews Sentiment Analyzer classifies Movie Reviews into positive and negative sentiments. It uses text analysis, natural language processing, machine learning techniques to predict the sentiment class. This solution is built around specialized vocabulary encountered in user reviews. It automates the manual effort to analyze movie reviews.

### Prerequisite

To run this algorithm you need to have access to the following AWS Services:
- Access to AWS SageMaker and the model package.
- An S3 bucket to specify input/output.
- Role for AWS SageMaker to access input/output from S3.

This sample notebook shows you how to deploy Mphasis HyperGraf Movie Reviews Sentiment Analyzer using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to Mphasis HyperGraf Movie Reviews Sentiment Analyzer. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

#### Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Output Result](#D.-Output-Result)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page Mphasis HyperGraf Movie Reviews Sentiment Analyzer
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
model_package_arn='arn:aws:sagemaker:us-east-2:786796469737:model-package/movie-reviews-sentiment-v1'

In [2]:
import pandas as pd
import numpy as np
import json
import os
import boto3
from zipfile import ZipFile
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
from IPython.display import Image, display

In [3]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket=sagemaker_session.default_bucket()
bucket

'sagemaker-us-east-2-786796469737'

### 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [25]:
model_name='movie-reviews-sentiment'

content_type='text/csv'

real_time_inference_instance_type='ml.m5.large'
batch_transform_inference_instance_type='ml.m5.large'

#### A. Create an endpoint

In [5]:
def predict_wrapper(endpoint, session):
    return sage.predictor.RealTimePredictor(endpoint, session,content_type)

#create a deployable model from the model package.

model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

In [6]:
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

-------------!

The class RealTimePredictor has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Once endpoint has been created, you would be able to perform real-time inference.

#### B. Create input payload

#### Instructions

    1) The input dataset should be in csv format.

    2) The column names in input file should be:

        *  Text: User Movie Reviews.

    3) input file should not contain more than 20 reviews. i.e input file will process top 20 rows.

In [40]:
file_name = 'sample_data.csv'

In [41]:
input_df = pd.read_csv(file_name)
input_df.head()

,Unnamed: 0,Text
0,0,This is one of my two or three favorite Stooge...
1,1,"When the Romulans come, they will not be beari..."
2,2,I have to confess that I am severely disappoin...
3,3,First off I want to say that I lean liberal on...
4,4,I was excited to see a sitcom that would hopef...


#### C. Perform real-time inference

In [10]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name $model_name --body fileb://$file_name --content-type 'text/csv' --region us-east-2 output.zip

{
    "ContentType": "application/zip",
    "InvokedProductionVariant": "AllTraffic"
}


#### D. Output Result

- The output file ('output.zip') contains the csv file with the list of movie reviews along with predicted sentiment.

In [11]:
file_path = os.getcwd()
output_df_path = os.path.join(file_path, 'output_df.csv')
print("Sample Output:")
output_df = pd.read_csv(output_df_path)
output_df.head()

Sample Output:


,Text,predicted_sentiment
0,This is one of my two or three favorite Stooge...,Positive
1,"When the Romulans come, they will not be beari...",Positive
2,I have to confess that I am severely disappoin...,Negative
3,First off I want to say that I lean liberal on...,Negative
4,I was excited to see a sitcom that would hopef...,Negative


#### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [12]:
predictor=sage.predictor.Predictor(model_name, sagemaker_session,content_type)
predictor.delete_endpoint(delete_endpoint_config=True)

### 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [ ]:
#upload the batch-transform job input files to S3
transform_input_folder = "data/input/batch"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name) 
print("Transform input uploaded to " + transform_input)

In [ ]:
#Run the batch-transform job
transformer = model.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

In [33]:
#output is available on following path
transformer.output_path

's3://sagemaker-us-east-2-786796469737/movie-reviews-sentiment-v1-2021-05-18-02-58-46-058'

In [34]:
s3_conn = boto3.client("s3")
bucket_name="sagemaker-us-east-2-786796469737"
with open('output.zip', 'wb') as f:
    s3_conn.download_fileobj(bucket_name, os.path.basename(transformer.output_path)+'/sample_data.csv.out', f)
    print("Output file loaded from bucket")

Output file loaded from bucket


In [35]:
file_path = os.getcwd()
file_name = 'output.zip'

file_object = open(file_name,'rb')
z = ZipFile(file_object)
file_names = []
for name in z.namelist():
    z.extract(name,file_path)
    file_names.append(name)
file_object.close()

output_folder_path = os.path.join(file_path, "output")

In [36]:

output_df_path = os.path.join(output_folder_path, 'output_df.csv')
print("Sample Output:")
output_df = pd.read_csv(output_df_path)
output_df.head()

Sample Output:


,Text,predicted_sentiment
0,This is one of my two or three favorite Stooge...,Positive
1,"When the Romulans come, they will not be beari...",Positive
2,I have to confess that I am severely disappoin...,Negative
3,First off I want to say that I lean liberal on...,Negative
4,I was excited to see a sitcom that would hopef...,Negative


### 4. Clean-up

#### A. Delete the model

In [37]:
model.delete_model()

#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.